<a href="https://colab.research.google.com/github/minddo/adversarial-machine-learning-at-scale/blob/master/Generate%20adversarial%20example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def FGSM(model, CRITERION ,images, labels, epsilon=None ,train= True):
    # Auther suggest that set up the epsilon randomly ~ N(0, 8) (range(0,16)), in test time, use the user defined  valus. 
    if train==True:
        epsilon=abs( torch.normal(torch.tensor([0.0]),torch.tensor([8/256]))).to("cuda")
    else:
        epsilon=epsilon.to("cuda")
    
    # as y_target, FGSM use ture label 
    labels = labels.detach()
    images.requires_grad = True
    outputs = model(images)
    
    # gradinet of y_true and output
    model.zero_grad()
    cost = CRITERION(outputs, labels)
    cost.backward()

    adv_images = images -  epsilon*images.grad.sign()
    # Adding clipping to maintain [0,1] range
    adv_image = torch.clamp(adv_images, 0, 1)
    return adv_images

In [0]:
def one_step_target(model, CRITERION ,images , epsilon=None ,train= True):
    
    # Auther suggest that set up the epsilon randomly ~ N(0, 8) (range(0,16)), in test time, use the user defined  valus. 
    if train==True:
        epsilon=abs( torch.normal(torch.tensor([0.0]),torch.tensor([8/256]))).to("cuda")
    else:
        epsilon=epsilon.to("cuda")
    
    # as y_target, step l.l use y_LL (least likely class)
    outputs = model(images)
    _, labels = torch.min(outputs.data, 1)
    labels = labels.detach_()
    
    images.requires_grad = True
    outputs = model(images)
    
    
    # gradinet of y_LL and output
    model.zero_grad()
    cost = CRITERION(outputs, labels)
    cost.backward()

    adv_images = images -  epsilon*images.grad.sign()
    # Adding clipping to maintain [0,1] range
    adv_image = torch.clamp(adv_images, 0, 1)
    return adv_images
    

In [0]:
def basic_iterative(model, CRITERION, images, labels, scale, alpha=1, iters=0, eps=None, train= True) :
    
    images = images.to("cuda")    
    outputs = model(images)
    _, labels = torch.min(outputs.data, 1)
    labels = labels.detach_()
        
    clamp_max = 255
    
     # Auther suggest that set up the epsilon randomly ~ N(0, 8) (range(0,16)), in test time, use the user defined  valus.
    if train ==True:
        eps= abs(np.random.normal(0, 8, 1)[0])
    else:
        eps=eps
        
    if iters == 0 :
    # The paper said min(eps + 4, 1.25*eps) is used as iterations
        iters = int(min(eps + 4, 1.25*eps))
        
    if scale :
        eps = eps / 255
        clamp_max = clamp_max / 255

    for i in range(iters) :    
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = CRITERION(outputs, labels).to("cuda")
        cost.backward()

        attack_images = images + alpha*images.grad.sign()
        
        # Clip attack images(X')
        # min{255, X+eps, max{0, X-eps, X'}}
        # = min{255, min{X+eps, max{max{0, X-eps}, X'}}}
        
        # a = max{0, X-eps}
        a = torch.clamp(images - eps, min=0)
        # b = max{a, X'}
        b = (attack_images>=a).float()*attack_images + (a>attack_images).float()*a
        # c = min{X+eps, b}
        c = (b > images+eps).float()*(images+eps) + (images+eps >= b).float()*b
        # d = min{255, c}
        images = torch.clamp(c, max=clamp_max).detach_()
            
    return images

In [0]:
def iterative_least_likely_class(model, CRITERION, images, scale, alpha=1, iters=0, eps=None, train= True) :
    
    images = images.to("cuda")    
    outputs = model(images)
    _, labels = torch.min(outputs.data, 1)
    labels = labels.detach_()
        
    clamp_max = 255
    
     # Auther suggest that set up the epsilon randomly ~ N(0, 8) (range(0,16)), in test time, use the user defined  valus.
    if train ==True:
        eps= abs(np.random.normal(0, 8, 1)[0])
    else:
        eps=eps
        
        
    if iters == 0 :
    # The paper said min(eps + 4, 1.25*eps) is used as iterations
        iters = int(min(eps + 4, 1.25*eps))
        
    if scale :
        eps = eps / 255
        clamp_max = clamp_max / 255
        
    for i in range(iters) :    
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = CRITERION(outputs, labels).to("cuda")
        cost.backward()

        attack_images = images - alpha*images.grad.sign()
        
        # Clip attack images(X')
        # min{255, X+eps, max{0, X-eps, X'}}
        # = min{255, min{X+eps, max{max{0, X-eps}, X'}}}
        
        # a = max{0, X-eps}
        a = torch.clamp(images - eps, min=0)
        # b = max{a, X'}
        b = (attack_images>=a).float()*attack_images + (a>attack_images).float()*a
        # c = min{X+eps, b}
        c = (b > images+eps).float()*(images+eps) + (images+eps >= b).float()*b
        # d = min{255, c}
        images = torch.clamp(c, max=clamp_max).detach_()
    
    return images